In [6]:
import pymysql
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import lxml
import datetime

In [11]:
class Data_gathering:
    def __init__(self):
        #defining url-s with data sources
        self.url_lotto = 'http://megalotto.pl/najwyzsze-wygrane/lotto'
        self.url_polish_cities = 'https://www.polskawliczbach.pl/Miasta'
        self.url_polish_provinces = 'https://www.polskawliczbach.pl/Wojewodztwa'      
        user = str(input('Enter SQL user name: '))
        password = str(input('Enter password: '))
        database = str(input('Enter SQL database name: '))
        self.conn = pymysql.connect("localhost", user, password, database)
        self.c = self.conn.cursor()
        print('Connected to SQL', database, 'database')
        
    def scrape_and_add_lottery_data(self):
        '''Scrapes lottery data (money prize, winner location, date) from http://megalotto.pl/najwyzsze-wygrane/lotto
        page and adds these data into SQL database'''
        
        #definig lists that will serve as a base for creating the table with prize, location and dates of lottery wins
        self.lottery_prizes_list = []
        self.lottery_locations_list = []
        self.lottery_dates_list = []

        while True:
            self.page = requests.get(self.url_lotto)
            html_content = BeautifulSoup(self.page.content, 'html.parser')
            prizes = html_content.find_all(class_ = 'numbers_in_list numbers_in_list_najwyzsze_wygrane')
            cities = html_content.find_all(class_ = 'date_in_list date_in_list_najwyzsze_wygrane_miasto')
            dates = html_content.find_all(class_='date_in_list date_in_list_najwyzsze_wygrane_date')
            next_page = html_content.find_all(class_ = 'prev_next')

            for index, prize in enumerate(prizes):
                #preparing prizes as int type. replace function was used to remove spaces prior convertion into int.
                if index > 0:
                    self.lottery_prizes_list.append(int(str(prizes[index]).split('>')[1].split(',')[0].replace(" ", "")))
                    self.lottery_locations_list.append(str(cities[index]).split('>')[1].split(' <')[0])
                    self.lottery_dates_list.append(str(dates[index]).split('>')[1].split(' <')[0])


            if "Następny" in str(next_page[1]): #getting url of the next page with prizes
                self.url_lotto = "http://megalotto.pl" + str(next_page[1]).split('href="')[1].split('"')[0]
            else:
                break #ending the 'while True' loop when there is no next page with prizes list

        self.c.execute("create table lottery_data ("
                       "prize_id int primary key auto_increment,"
                       "lottery_prize int,"
                       "lottery_location varchar(255),"
                       "lottery_date date)")
        self.conn.commit()
        print("lottery_data table was added to the project_lotto database")
        
        for index, prize in enumerate(self.lottery_prizes_list):
            self.c.execute("insert into lottery_data values (default, %s, %s, %s)",
                           (prize, self.lottery_locations_list[index], self.lottery_dates_list[index]))
        self.conn.commit()
        print("lottery data were added to lottery_data table")
        self.conn.close()

In [12]:
lotto = Data_gathering()

Enter SQL user name:  piotrsoczewka
Enter password:  qwerty
Enter SQL database name:  lotto


Connected to SQL lotto database


In [13]:
lotto.scrape_and_add_lottery_data()

lottery_data table was added to the project_lotto database
lottery data were added to lottery_data table
